# Model Selection

In this notebook we will test different models to find the one that gives us the best results.

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def f1_scores(Z, y_test):
    averages = ['macro', 'micro', 'weighted']
    for avg in averages:
        score = f1_score(Z, y_test, average=avg)
        print("f1 score ({}): {}".format(avg, score))

def test_model(X, y, model_name, model):
    print("MODEL: {}".format(model_name))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    Z = model.predict(X_test)
    f1_scores(Z, y_test)
    
def testModelKFold(X, y, model_name, model, k):
    print("CROSS VALIDATION FOR: {}".format(model_name))
    skf = StratifiedKFold(n_splits=5)
    skf.get_n_splits(X, y)
    f1_scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        Z = model.predict(X_test)
        f1_scores.append(f1_score(Z, y_test))
    print("f1 scores: {}".format(f1_scores))
    print("MEAN: {}".format(np.mean(f1_scores)))

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

models = {
    "Logistic Regression": LogisticRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors = 3),
    "GaussianNB": GaussianNB(),
    "Perceptron": Perceptron(),
    "SGDClassifier": SGDClassifier(),
    "Decision Tree": DecisionTreeClassifier()   
}

dataset_path = "data.csv"
data = pd.read_csv(dataset_path, sep=";")
X = data.drop("diagnosis", axis=1).values
y = data["diagnosis"]

for model in models:
    test_model(X, y, model, models[model])
    testModelKFold(X, y, model, models[model], k=5)
    print('')

MODEL: Logistic Regression
f1 score (macro): 0.7854557933487241
f1 score (micro): 0.8034026465028355
f1 score (weighted): 0.8029334477275646
CROSS VALIDATION FOR: Logistic Regression
f1 scores: [0.8554913294797688, 0.8082595870206489, 0.5481727574750831, 0.7988668555240793, 0.4453441295546559]
MEAN: 0.6912269318108473

MODEL: SVC
f1 score (macro): 0.8605929020102974
f1 score (micro): 0.8733459357277883
f1 score (weighted): 0.8732662292670539
CROSS VALIDATION FOR: SVC
f1 scores: [0.8913043478260869, 0.8385269121813033, 0.7223382045929019, 0.8825214899713467, 0.5522388059701493]
MEAN: 0.7773859521083576

MODEL: Random Forest
f1 score (macro): 0.880633364456894
f1 score (micro): 0.8903591682419659
f1 score (weighted): 0.8904879868351457
CROSS VALIDATION FOR: Random Forest
f1 scores: [0.972972972972973, 0.8604651162790697, 0.7056530214424951, 0.8603988603988604, 0.5358490566037736]
MEAN: 0.7870678055394345

MODEL: K-Nearest Neighbors
f1 score (macro): 0.8245522600208471
f1 score (micro): 0

We can check the confusion matrix for the Random Forest model.

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
Z = model.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, Z))

print("\nClassfication Report")
print(classification_report(y_test, Z))

Confusion Matrix
[[319  32]
 [ 25 153]]

Classfication Report
             precision    recall  f1-score   support

          0       0.93      0.91      0.92       351
          1       0.83      0.86      0.84       178

avg / total       0.89      0.89      0.89       529

